<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-4/notebooks/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Setup

In [ ]:
!pip install pyspark graphframes ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=44e676ff0b7aa19efb251654ebe916f56adabb0231e291f87915455fbc11c924
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
%%bash
java --version

openjdk 11.0.22 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


In [ ]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.22
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.


# Imports

In [ ]:
import os
import sys
import json
import pandas as pd
from itertools import combinations
import math
import statistics
from pyspark import SparkContext
import numpy as np

from pprint import pprint

%load_ext autotime

time: 350 µs (started: 2024-03-28 05:39:50 +00:00)


In [ ]:
!spark-shell --packages graphframes:graphframes:0.8.3-spark3.5-s_2.12

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-118b952b-7c11-496f-a207-180eb284b4ad;1.0
	confs: [default]
	found graphframes#graphframes;0.8.3-spark3.5-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.3-spark3.5-s_2.12/graphframes-0.8.3-spark3.5-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.3-spark3.5-s_2.12!graphframes.jar (74ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (45ms)
:: resolution report :: resolve 1541ms :: artifacts dl 125ms
	:: modules in use:
	graphfr

# Configuration

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/hw4")


class Path:
    curren_dir = os.getcwd()
    input_csv_file = os.path.join(curren_dir, "ub_sample_data.csv")

    output_dir = os.path.join(curren_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    task1_output = os.path.join(output_dir, "task1_op.txt")
    task2_output = os.path.join(output_dir, "task2_op.txt")

time: 426 ms (started: 2024-03-28 05:40:27 +00:00)


# Task 1

In [ ]:
# %%writefile task1.py
import time
import os
import sys

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, DataFrame
from graphframes import GraphFrame
from itertools import combinations
from pyspark.sql.functions import udf, col, size, array_intersect
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StructType, StructField, StringType


os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.3-spark3.5-s_2.12 pyspark-shell"


def read_csv(spark: SparkSession, file_path: str):
    """
    Read CSV file into DataFrame
    """
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    return df


def construct_graph(spark: SparkSession, df, filter_threshold: int):
    """
    Function to construct the social network graph.
    """
    # Create user to business map
    user2bus = (
        df.groupBy("user_id")
        .agg({"business_id": "collect_set"})
        .withColumnRenamed("collect_set(business_id)", "business_set")
    )

    # Create edge dataframe
    edges = (
        user2bus.alias("u1")
        .join(user2bus.alias("u2"), col("u1.user_id") != col("u2.user_id"))
        # Filter edges by threshold
        .where(size(array_intersect(col("u1.business_set"), col("u2.business_set"))) >= filter_threshold)
        .select(col("u1.user_id").alias("src"), col("u2.user_id").alias("dst"))
        .distinct()
    )

    # Create nodes DataFrame
    nodes_src = edges.select(col("src").alias("id"))
    nodes_dst = edges.select(col("dst").alias("id"))
    nodes = nodes_src.union(nodes_dst).distinct()

    # Construct GraphFrame
    graph = GraphFrame(nodes, edges)

    return graph


def detect_communities(graph):
    """
    Function to detect communities using Label Propagation Algorithm.
    """
    # Use the Label Propagation Algorithm to detect communities
    communities = graph.labelPropagation(maxIter=5)
    return communities


def save_communities(communities, output_file_path):
    """
    Function to save communities to a txt file.
    """
    # Sort communities by size and then by lexicographical order
    sorted_communities = (
        communities
        .rdd
        .map(lambda x: (x[1], x[0]))
        .groupByKey()
        .map(lambda x: sorted(list(x[1])))
        .sortBy(lambda x: (len(x), x))
        .collect()
    )

    # Save communities to the output file
    with open(output_file_path, 'w') as file:
        for community in sorted_communities:
            file.write(", ".join(community) + "\n")


def task1(filter_threshold: int, input_file_path: str, community_output_file_path: str):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 1").set("spark.jars.packages", "graphframes:graphframes:0.8.3-spark3.5-s_2.12")
    sc = SparkContext(conf=conf).getOrCreate()
    spark = SparkSession(sparkContext=sc)
    spark.sparkContext.setLogLevel("ERROR")

    try:
        start_time = time.time()

        df = read_csv(spark, input_file_path)

        # Construct graph
        graph = construct_graph(spark, df, filter_threshold)

        # Detect Communities
        communities = detect_communities(graph)
        # communities.show()

        # Save the generated communities data
        save_communities(communities, community_output_file_path)

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     # Check if the correct number of command-line arguments are provided
#     if len(sys.argv) != 4:
#         print(
#             "Usage: spark-submit --packages graphframes:graphframes:0.8.2-spark3.1-s_2.12 task1.py <filter threshold>"
#             " <input file path> <community output file path>"
#         )
#         sys.exit(1)

#     # Parse command-line arguments
#     filter_threshold = int(sys.argv[1])
#     input_file_path = sys.argv[2]
#     community_output_file_path = sys.argv[3]

#     # Call task1 function
#     task1(filter_threshold, input_file_path, community_output_file_path)

task1(2, Path.input_csv_file, Path.task1_output)

Duration: 114.79595375061035

time: 1min 56s (started: 2024-03-28 06:10:58 +00:00)


In [ ]:
# !spark-submit --packages graphframes:graphframes:0.8.3-spark3.5-s_2.12 task1.py 2 ub_sample_data.csv t1.txt
!cat output/task1_op.txt

1UR-fzasIATn6hoYZein4g
3Vd_ATdvvuVVgn_YCpz8fw
3xESvNr3bkJikpjLRqcBcA
5AlOIOMTzNhU1tSOtQ6mDw
6pNftAvuCGpkaXBgL-4QdQ
8EPHywVzuKnevp2zAzbMug
Cf0chERnfd06ltnN45xLNQ
Ksp1e9Dw0Jcog_ZBD3-45g
KtNGMeHmCpagrTTqcfBcgA
PTcKYk_qU-WzbQjraE3TgQ
RqTl3f3ZIhrjILKJFCEKDw
_m1ot2zZetDgjerAD2Sidg
abwO4VF4JcVYQ7CURAwsyw
bg9FRc71Daub48GmIKfX_A
fccMIyBd6ZhFUVAhvSqsrw
kteNzDbDmsatpOC0xssmew
lFFIpfkox55FQ-wZD6BBVw
lL-wNa0TKK6LXrlcVmjYrQ
nrUQRMx6K-KJnoC15rMO2g
s-QuErK1oRlAzT5qnO6FOQ
vENR70IrUsDNTDebbuxyQA
4hiLotPELiIRugUtWC9L6A, Xwnf20FKuikiHcSpcEbpKQ
5AKt52TW50eWHk9Ia03uzg, swx0FJkZWb6nVP58ZsMl7w
9W73B44Iw8WslrTNB2CdCg, LZoZfNY8HNlIqJlNqeYVmw
JDEhGFIPAyIMkfHD0G13UA, bjFG1IviPp_9Ae9WwAvHqg
52MlnjkvSLLqKKSRrbrH0w, GqJRlbga27rUpolN-SP9Ig, fkMiipV4j_DG5nSoBnZvbg
-ki-qIZHP4zZ2D49-b8ZzA, 3gq1d-Qiyajk5TWq9jNytA, 8pLFYhWKzILEIYSl5qE-3w, Ahl7wBbav9ZoBrvwi3vyew, P3B7FfBpXERa-KdxwIGkHA, yvMQhKw2_dq-bgf3lOCjEg
0o8HUzggoNKay9-ZMj3HqQ, 3egcdazws_x1wW35jgXfNw, QEkgYrrUEPJJbiUCLqf0fg, aNmrSw98PhdvZkLgxP2PWg, g8OozJjZwliRYdAZOTI